# Chapter 9 Solvers, templates, and implied volatilities

In this chapter we will be covering the implementation of two techniques for solving numerically equations the bisection and the Newton-Raphson. Both techniques are implemented originally in the book using templates (to add flexibility to the code). However since we are coding in python we do not need templates and we can use directly functions.

The objective of these solvers is to get the implied volatility given a model, the price of the derivative and the rest of variables.

In [3]:
import numpy as np
import scipy.stats as si
from math import exp,log,sqrt,pi

We will define the BlackSchools call (with the BS model for pricing, obviously).

In [4]:
class BSCall:
    def __init__(self,r_,d_,T_,Spot_,Strike_):
        self.__r = r_
        self.__d = d_
        self.__T = T_
        self.__Spot = Spot_
        self.__Strike = Strike_
        
    def __call__(self,Vol):
        d1 = (log(self.__Spot / self.__Strike) + (self.__r + 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
        d2 = (log(self.__Spot / self.__Strike) + (self.__r - 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
    
        return (self.__Spot * exp(-self.__d * self.__T) * si.norm.cdf(d1, 0.0, 1.0) - self.__Strike * 
                exp(-self.__r * self.__T) * si.norm.cdf(d2, 0.0, 1.0))

We check that it works properly.

In [5]:
theCall = BSCall(0.05, 0, 0.25, 50, 50)
theCall(0.41)

4.37472312788827

Now we implement the bisection. This technique gets an interval divides it in halfs checking if in each side the dependent variable is negative or positive ( hoping to have in one extreme a negative result and in the other a positive one). If that happens it means that in the middle of that interval the function is 0 (thus the solution is there), only if the function is continious.

This is property of continious functions is call Bolzano's theorem:
https://en.wikipedia.org/wiki/Intermediate_value_theorem

Once that the interval is bisected the condition is evaluated, and the bisection is done again and again until it reaches convergency. To get a more profound explanation check the original book or the following link:
https://en.wikipedia.org/wiki/Bisection_method

In [6]:
def Bisection(Target,Low,High,Tolerance,TheFunction):
    x = 0.5*(Low+High)
    y = TheFunction(x)
    
    while abs(y-Target) > Tolerance:
        if y < Target:
            Low = x
            
        elif y > Target:
            High = x
            
        x = 0.5*(Low + High)
        
        y = TheFunction(x)
        
        
    return x

We check that it works to get the volatility.

In [19]:
theCall = BSCall(0.05, 0, 0.25, 50, 50)

In [35]:
%%time
Bisection(4.37472312788827,0.01,0.6,0.0000000001,theCall)

Wall time: 15.6 ms


0.4100000000066939

This volatility is correct.

In [8]:
theCall(Bisection(4.37472312788827,0.01,0.6,0.0000000001,theCall))

4.3747231279541445

For the Newton-Raphson we use the derivative of the original function to reach to the convergence faster. This has advantages such as the speed however if the initial values are not well inputted then it is impossible to reach convergency, it is much more unstable than the bisection.

Check the following link to learn more about the Newton-Raphson method: https://en.wikipedia.org/wiki/Newton%27s_method

In [9]:
class BSCallTWO:
    def __init__(self,r_,d_,T_,Spot_,Strike_):
        self.__r = r_
        self.__d = d_
        self.__T = T_
        self.__Spot = Spot_
        self.__Strike = Strike_
        
    def Price(self,Vol):
        d1 = (log(self.__Spot / self.__Strike) + (self.__r + 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
        d2 = (log(self.__Spot / self.__Strike) + (self.__r - 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
    
        return (self.__Spot * exp(-self.__d * self.__T) * si.norm.cdf(d1, 0.0, 1.0) - self.__Strike * 
                exp(-self.__r * self.__T) * si.norm.cdf(d2, 0.0, 1.0))
    
    def Vega(self,Vol):
        d1 = (log(self.__Spot / self.__Strike) + (self.__r + 0.5 * Vol ** 2) * self.__T) / (Vol * sqrt(self.__T))
        return self.__Spot*exp(-0.5*d1*d1)*sqrt(self.__T)/sqrt(2*pi)
        

In [39]:
def NewtonRaphson(Target, Start, Tolerance, Value, Derivative):
    y = Value(Start)
    x = Start
    
    while abs(y - Target) > Tolerance:
        d = Derivative(x)
        x += (Target - y)/d
        y = Value(x)  
        
        
    return x

As we can see we get the same result but faster.

In [40]:
theCall = BSCallTWO(0.05, 0, 0.25, 50, 50)

In [54]:
%%time
NewtonRaphson(4.37472312788827,0.5,0.00000000001,theCall.Price,theCall.Vega)

Wall time: 0 ns


0.4100000000000002

**Exercise 9.1** Modify the Newton-Raphson routine so that it does not endlessly loop if a root is not found.

For this exercise we will use the design pattern strategy, in order to externalize how part of one algorithm is implemented, using one of our terminator classes we developed previously in chapter 5.

In [56]:
import time

class TerminatorMC:
    def __init__(self):
        pass
 
    def GetTerminatorCondition(self):
        pass
    
    def clone(self):
        pass
    
    def __del__(self):
        pass   

class TerminatorRunTime(TerminatorMC):
    def __init__(self,MaxTime_):
        self.__MaxTime = MaxTime_
        self.__TotalTime = 0
        self.__InitTime = -1
    
    def GetTerminatorCondition(self):
        if self.__InitTime == -1:
            self.__InitTime = int(round(time.time() * 1000))
            return False
        else:
            self.__TotalTime += int(round(time.time() * 1000)) - self.__InitTime
            
            if self.__TotalTime >= self.__MaxTime:
                return True
        
            else:
                self.__InitTime = int(round(time.time() * 1000))
                return False
        
    def clone(self):
        return deepcopy(self)
    
    def __del__(self):
        del self

In [241]:
def NewtonRaphson(Target, Start, Tolerance, Value, Derivative, terminator):
    y = Value(Start)
    x = Start
    
    while abs(y - Target) > Tolerance:
        d = Derivative(x)
        x += (Target - y)/d
        y = Value(x)  
        if terminator.GetTerminatorCondition() == True:
            print('Terminator activated, exiting the solver')
            break
        else: 
            pass
        
        
    return x

In [240]:
%%time 
terminator = TerminatorRunTime(2)
print(NewtonRaphson(4.37472312788827,0.5,0.00000000001,theCall.Price,theCall.Vega, terminator))

0.4100000000000002
Wall time: 0 ns


In [242]:
%%time 
terminator = TerminatorRunTime(0.00000000000000000001)
print(NewtonRaphson(4.37,0.05,0.00000000000000000000000000000000000000000001,theCall.Price,theCall.Vega, terminator))

Terminator activated, exiting the solver
0.40952006656124634
Wall time: 15.6 ms


**Exercise 9.2** Take your favourite numerical integration routine, e.g. the trapezium rule, and write a template routine to carry it out.

For this exercise we will code the trapezium rule. Find more information here: https://en.wikipedia.org/wiki/Trapezoidal_rule

In [12]:
def TrapezodialRule(Start, Finish, Intervals, Value):
    delta = (Finish - Start) / (Intervals)
    x = np.arange(Start, Finish + delta, delta)
    y = list(map(Value,x))
    total = y.pop(0)
    total += y.pop(-1)
    total += 2*sum(y)
    total *= delta/2
    return total

In [13]:
def quadratic(x):
    return 3*(x**2) - 2*x + 1

In [14]:
TrapezodialRule(0,5,100,quadratic)

105.00625000000002

We can also use a modification of the trapezodial rule without specifying how many intervals, just the Tolerance.

In [15]:
def TrapezodialRule(Start, Finish, Tolerance, Value):
    prev = -1
    Intervals = 2
    total = 0
    while abs(prev  - total) > Tolerance:
        prev = total
        delta = (Finish - Start) / (Intervals)
        x = np.arange(Start, Finish + delta, delta)
        y = list(map(Value,x))
        total = y.pop(0)
        total += y.pop(-1)
        total += 2*sum(y)
        total *= delta/2
        Intervals*=2
    return total

In [16]:
TrapezodialRule(0,5,0.000001,quadratic)

105.00000023283064

We could even use the strategy pattern to define why the routine stops, using a terminator object that can handle different conditions to stop such as the Tolerance or the runtime.

**Exercise 9.3** Write a routine to price a vanilla option by MonteCarlo or trees where the pay-off is passed in as a template parameter expressed via a function object.

Since there are no templates in python (because it is not a typed language) it does not make sense to do this exercise.